# Export XRF Fits to png images

This notebook will export xrf fits of mapping scans as png images.
The scans can either be single scans, or a set of scans taken across a sample to reduce the sample files.
All the scans should be from the same sample and same region, with a small overlap in motor positions.
(Basically a set of scans created by the mapping tool for a single region)

*Remember to run XRF_fitting.ipynb first, this notebook wil not run otherwise!!*

#### Note that the scans *must* be given in list form, even if it is a single scan!
The list should ideally come fromthe scan database

In [ ]:
#Import necessary libaries

%matplotlib widget
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import h5py
#To import DanMAX from the folder above:
sys.path.append('../')
import DanMAX as DM
from lib.xrf_pyMCA_fit import xrfBatch
style = DM.darkMode(style_dic={'figure.figsize':'large'})

def _limit_map(im,minmax):
    im[im<minmax[0]] = minmax[0]
    im[im>minmax[1]] = minmax[1]
    return (im-minmax[0])/(minmax[1]-minmax[0])

def _save_im(im,save_folder,save_file):
                if not os.path.isdir(save_folder):
                    os.makedirs(save_folder)
                    os.chmod(save_folder,0o770)
                plt.imsave(save_file,im)
                os.chmod(save_file,0o770)

In [ ]:
#Select scans to fit, must be a list!!!
groups = DM.getProposalScans()
#Define parameters to find the data
proposal,visit = DM.getCurrentProposal() #Default will be collected from the current path, you can set others, if you wish to use data from a previous beamtime
proposal_type,beamline = DM.getCurrentProposalType()

#Sessionpath will tell the fitter which proposal and visit it should read data from
#This will also be used to find where to store fitted data
sessionpath = f'/data/{proposal_type}/{beamline}/{proposal}/{visit}/' #Default will be f'/data/visitors/danmax/{proposal}/{visit}/'

## Export images to png files
The following code snippits takes the fitted maps from above and exports them as images.

In [ ]:
#Define the information needed to export the fits

#Define which maps to plot, the lists give the lower and upper limit.
#Set limits to "None" for default value
elements_to_export = {'Ca_K':{'color': 'R','limit':[20, 700]},
               'Sr_K': {'color': 'R','limit': [20, 220]},
               'Zn_K': {'color': 'R','limit': [10, 150]},
                }

#Set elements to be combined before exporting, will be R,G,B, and use limits from above
elements_to_combine = [['Ca_K','Sr_K','Zn_K']]
#Set the base path in the h5 fit file to the fit maps
h5_fit_path = 'xrf_fits/xrf_fit/results/parameters/'

#Set the location of the scalebar 
scale_place = [[10,15],[10,49]]

In [ ]:

for group in groups:
    print(f'Making figures for group: {group}')
    for sample in groups[group]:
        print(f'\tMaking figures for sample: {sample}')
        scans = groups[group][sample]
        fit_folder, fit_file = DM.mapping.getXRFFitFilename(scans,proposal_type,beamline,proposal,visit)
        xrf_file = f'{fit_folder}/elements/{fit_file}'
        for element in elements_to_export:
            with h5py.File(xrf_file,'r') as xfile:
                element_map= _limit_map(xfile[f'{h5_fit_path}{element}'][:],elements_to_export[element]['limit'])
                
            if elements_to_export[element]['color'] in 'Red':
                elmap = DM.mapping.combineMaps(element_map,np.zeros(element_map.shape),np.zeros(element_map.shape), scale_place=scale_place)
            elif elements_to_export[element]['color'] in 'Green':
                elmap = DM.mapping.combineMaps(np.zeros(element_map.shape),element_map,np.zeros(element_map.shape), scale_place=scale_place) 
            elif elements_to_export[element]['color'] in 'Blue':
                elmap = DM.mapping.combineMaps(np.zeros(element_map.shape),np.zeros(element_map.shape),element_map, scale_place=scale_place)
            save_folder = f'/data/{proposal_type}/{beamline}/{proposal}/{visit}/process/xrf_fit_images/{group}_individual'
            save_file = f'{save_folder}/{group}_{sample}_{element}.png'
            _save_im(elmap,save_folder,save_file)
        for combination in elements_to_combine:
            with h5py.File(xrf_file,'r') as xfile:
                red_map = _limit_map(xfile[f'{h5_fit_path}{combination[0]}'][:],elements_to_export[combination[0]]['limit'])
                green_map = _limit_map(xfile[f'{h5_fit_path}{combination[1]}'][:],elements_to_export[combination[1]]['limit'])
                blue_map = _limit_map(xfile[f'{h5_fit_path}{combination[2]}'][:],elements_to_export[combination[2]]['limit'])
            elmap = DM.mapping.combineMaps(red_map,green_map,blue_map,scale_place=scale_place)
            save_folder = f'/data/{proposal_type}/{beamline}/{proposal}/{visit}/process/xrf_fit_images/{group}_combination'
            save_file = f'{save_folder}/{group}_{sample}_{combination[0]}_{combination[1]}_{combination[2]}.png'
            _save_im(elmap,save_folder,save_file)